### =====================================================================
### IMPORTACIÓN GENERAL DE LA INFORMACIÓN.
### =====================================================================

In [284]:
## IMPORTACIÓN GENERAL DE LIBRERIAS Y VISUALIZACIÓN DE DATOS (matplotlib y seaborn)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as DT
import warnings
import descartes
import geopandas as gpd
import json
import requests
import geocoder

from sklearn.ensemble import RandomForestRegressor
from shapely.geometry import Point, Polygon
from urllib.request import urlopen #Python3

%matplotlib inline
warnings.filterwarnings('ignore')
plt.style.use('default') 
sns.set(style="whitegrid") 
plt.rcParams['figure.figsize'] = (15, 10)

In [285]:
#lEEMOS EL CSV
train = pd.read_csv('../../DATA_TP2/train.csv')
test = pd.read_csv('../../DATA_TP2/test.csv')
respuesta = pd.read_csv('../../DATA_TP2/ejemploRespuesta.csv')

### =====================================================================
### PREPARACIÓN DE DATOS, FEATURE ENGINEERING.
### =====================================================================

### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [286]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 23 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
direccion                     186928 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
idzona                        211379 non-null float64
lat                           116512 non-null float64
lng                           116512 non-null float64
fecha                         240

In [287]:
#Borramos las columnas que no nos interesan
train.drop('idzona', axis=1, inplace=True)
train.drop('direccion', axis=1, inplace=True)
train.drop('lat', axis=1, inplace=True)
train.drop('lng', axis=1, inplace=True)

In [288]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 19 columns):
id                            240000 non-null int64
titulo                        234613 non-null object
descripcion                   238381 non-null object
tipodepropiedad               239954 non-null object
ciudad                        239628 non-null object
provincia                     239845 non-null object
antiguedad                    196445 non-null float64
habitaciones                  217529 non-null float64
garages                       202235 non-null float64
banos                         213779 non-null float64
metroscubiertos               222600 non-null float64
metrostotales                 188533 non-null float64
fecha                         240000 non-null object
gimnasio                      240000 non-null float64
usosmultiples                 240000 non-null float64
piscina                       240000 non-null float64
escuelascercanas              240

In [289]:
#Vemos cuales tienen NANs
train.isna().sum()

id                                0
titulo                         5387
descripcion                    1619
tipodepropiedad                  46
ciudad                          372
provincia                       155
antiguedad                    43555
habitaciones                  22471
garages                       37765
banos                         26221
metroscubiertos               17400
metrostotales                 51467
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
precio                            0
dtype: int64

In [290]:
#Borramos los nulls de las columnas categoricas que no nos interesan
train.dropna(subset=['tipodepropiedad', 'provincia', 'ciudad'], inplace=True)

#Consideramos que las propiedades con garages NAN no poseen y banos poseen 1
train['garages'].fillna(0, inplace=True)
train['banos'].fillna(1, inplace=True)

#Pruebo con el promedio general MODIFICAR!
train['antiguedad'].fillna(train['antiguedad'].mean(), inplace=True)
train['habitaciones'].fillna(train['habitaciones'].mean(), inplace=True)
train['metroscubiertos'].fillna(train['metroscubiertos'].mean(), inplace=True)
train['metrostotales'].fillna(train['metrostotales'].mean(), inplace=True)

In [291]:
#Compruebo que no hayan quedado NAN
train.isnull().sum()

id                               0
titulo                        5382
descripcion                   1575
tipodepropiedad                  0
ciudad                           0
provincia                        0
antiguedad                       0
habitaciones                     0
garages                          0
banos                            0
metroscubiertos                  0
metrostotales                    0
fecha                            0
gimnasio                         0
usosmultiples                    0
piscina                          0
escuelascercanas                 0
centroscomercialescercanos       0
precio                           0
dtype: int64

In [292]:
#Borro todas las categoricas para probar
train.drop('titulo', axis=1, inplace=True)
train.drop('descripcion', axis=1, inplace=True)
train.drop('tipodepropiedad', axis=1, inplace=True)
train.drop('ciudad', axis=1, inplace=True)
train.drop('provincia', axis=1, inplace=True)

In [293]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239600 entries, 0 to 239999
Data columns (total 14 columns):
id                            239600 non-null int64
antiguedad                    239600 non-null float64
habitaciones                  239600 non-null float64
garages                       239600 non-null float64
banos                         239600 non-null float64
metroscubiertos               239600 non-null float64
metrostotales                 239600 non-null float64
fecha                         239600 non-null object
gimnasio                      239600 non-null float64
usosmultiples                 239600 non-null float64
piscina                       239600 non-null float64
escuelascercanas              239600 non-null float64
centroscomercialescercanos    239600 non-null float64
precio                        239600 non-null float64
dtypes: float64(12), int64(1), object(1)
memory usage: 27.4+ MB


### =====================================================================
### TRATAMIENTO SOBRE TEST.
### =====================================================================

In [294]:
#Borramos las columnas que no nos interesan
test.drop('idzona', axis=1, inplace=True)
test.drop('direccion', axis=1, inplace=True)
test.drop('lat', axis=1, inplace=True)
test.drop('lng', axis=1, inplace=True)

In [295]:
#Vemos cuales tienen NANs
test.isna().sum()

id                                0
titulo                         1378
descripcion                     401
tipodepropiedad                   7
ciudad                           83
provincia                        42
antiguedad                    10714
habitaciones                   5628
garages                        9323
banos                          6554
metroscubiertos                4299
metrostotales                 12655
fecha                             0
gimnasio                          0
usosmultiples                     0
piscina                           0
escuelascercanas                  0
centroscomercialescercanos        0
dtype: int64

In [296]:
#Borramos los nulls de las columnas categoricas que no nos interesan
test.dropna(subset=['tipodepropiedad', 'provincia', 'ciudad'], inplace=True)

#Consideramos que las propiedades con garages NAN no poseen y banos poseen 1
test['garages'].fillna(0, inplace=True)
test['banos'].fillna(1, inplace=True)

#Pruebo con el promedio general MODIFICAR!!
test['antiguedad'].fillna(test['antiguedad'].mean(), inplace=True)
test['habitaciones'].fillna(test['habitaciones'].mean(), inplace=True)
test['metroscubiertos'].fillna(test['metroscubiertos'].mean(), inplace=True)
test['metrostotales'].fillna(test['metrostotales'].mean(), inplace=True)


In [297]:
#Compruebo que no hayan quedado NULL
test.isna().sum()

id                               0
titulo                        1378
descripcion                    393
tipodepropiedad                  0
ciudad                           0
provincia                        0
antiguedad                       0
habitaciones                     0
garages                          0
banos                            0
metroscubiertos                  0
metrostotales                    0
fecha                            0
gimnasio                         0
usosmultiples                    0
piscina                          0
escuelascercanas                 0
centroscomercialescercanos       0
dtype: int64

In [298]:
#Borro todas las categoricas para probar
test.drop('titulo', axis=1, inplace=True)
test.drop('descripcion', axis=1, inplace=True)
test.drop('tipodepropiedad', axis=1, inplace=True)
test.drop('ciudad', axis=1, inplace=True)
test.drop('provincia', axis=1, inplace=True)

In [299]:
#Para que ande el Random Forest

In [300]:
train['fecha'] = pd.to_datetime(train['fecha'], infer_datetime_format=True)
test['fecha'] = pd.to_datetime(test['fecha'], infer_datetime_format=True)
train['anio'] = train['fecha'].dt.year
test['anio'] = test['fecha'].dt.year
train.drop('fecha', axis=1, inplace=True)
test.drop('fecha', axis=1, inplace=True)


In [301]:
train.to_csv('DATA/train.csv', index=False)
test.to_csv('DATA/test.csv', index=False)

In [303]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59912 entries, 0 to 59999
Data columns (total 13 columns):
id                            59912 non-null int64
antiguedad                    59912 non-null float64
habitaciones                  59912 non-null float64
garages                       59912 non-null float64
banos                         59912 non-null float64
metroscubiertos               59912 non-null float64
metrostotales                 59912 non-null float64
gimnasio                      59912 non-null float64
usosmultiples                 59912 non-null float64
piscina                       59912 non-null float64
escuelascercanas              59912 non-null float64
centroscomercialescercanos    59912 non-null float64
anio                          59912 non-null int64
dtypes: float64(11), int64(2)
memory usage: 6.4 MB
